In [2]:
import numpy as np
import random
import time
import cllm_utils

# 获取名称
data = open("dinos.txt", "r", encoding='utf-8').read()

# 转化为小写字符
data = data.lower()

# 转化为无序且不重复的元素列表
chars = list(set(data))

# 获取大小信息
data_size, vocab_size = len(data), len(chars)

print(chars)
print("共计有%d个字符，唯一字符有%d个"%(data_size,vocab_size))


['重', '洞', '花', '走', '清', '马', '念', '家', '深', '天', '和', '以', '不', '字', '日', '助', '网', '女', '城', '是', '万', '落', '男', '根', '香', '长', '到', '李', '语', '东', '彩', '在', '观', '口', '瓜', '起', '水', '雨', '山', '田', '功', '成', '白', '为', '秀', '张', '丽', '红', '春', '事', '化', '十', '有', '居', '暖', '拔', '忘', '热', '安', '路', '心', '下', '无', '业', '罗', '千', '来', '别', '前', '灯', '绿', '地', '风', '开', '乐', '里', '结', '一', '自', '生', '由', '流', '火', '人', '军', '再', '\n', '往', '苗', '手', '鸟']
共计有149个字符，唯一字符有91个


In [3]:
char_to_ix = {ch:i for i, ch in enumerate(sorted(chars))}
ix_to_char = {i:ch for i, ch in enumerate(sorted(chars))}

print(char_to_ix)
print(ix_to_char)


{'\n': 0, '一': 1, '万': 2, '下': 3, '不': 4, '业': 5, '东': 6, '为': 7, '丽': 8, '乐': 9, '事': 10, '人': 11, '以': 12, '再': 13, '军': 14, '别': 15, '到': 16, '前': 17, '功': 18, '助': 19, '化': 20, '十': 21, '千': 22, '口': 23, '和': 24, '在': 25, '地': 26, '城': 27, '天': 28, '女': 29, '字': 30, '安': 31, '家': 32, '居': 33, '山': 34, '开': 35, '张': 36, '彩': 37, '往': 38, '心': 39, '忘': 40, '念': 41, '成': 42, '手': 43, '拔': 44, '无': 45, '日': 46, '春': 47, '是': 48, '暖': 49, '有': 50, '李': 51, '来': 52, '根': 53, '水': 54, '洞': 55, '流': 56, '深': 57, '清': 58, '火': 59, '灯': 60, '热': 61, '瓜': 62, '生': 63, '田': 64, '由': 65, '男': 66, '白': 67, '秀': 68, '红': 69, '结': 70, '绿': 71, '网': 72, '罗': 73, '自': 74, '花': 75, '苗': 76, '落': 77, '观': 78, '语': 79, '走': 80, '起': 81, '路': 82, '里': 83, '重': 84, '长': 85, '雨': 86, '风': 87, '香': 88, '马': 89, '鸟': 90}
{0: '\n', 1: '一', 2: '万', 3: '下', 4: '不', 5: '业', 6: '东', 7: '为', 8: '丽', 9: '乐', 10: '事', 11: '人', 12: '以', 13: '再', 14: '军', 15: '别', 16: '到', 17: '前', 18: '功', 19: '助', 20: '化', 21: '十',

In [4]:
def clip(gradients, maxValue):
    """
    使用maxValue来修剪梯度
    
    参数：
        gradients -- 字典类型，包含了以下参数："dWaa", "dWax", "dWya", "db", "dby"
        maxValue -- 阈值，把梯度值限制在[-maxValue, maxValue]内
        
    返回：
        gradients -- 修剪后的梯度
    """
    # 获取参数
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
    
    # 梯度修剪
    for gradient in [dWaa, dWax, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)

    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients


In [5]:
def sample(parameters, char_to_ix, seed):
    """
    根据训练好的model参数parameters，实际生成字符序列，第一个字符被初始化为全0的‘虚’编码投进去
    停止的条件是产生换行符或者生成的长度.50（设定的），实际上最终的就是参数。
    参数：
        parameters -- 包含了Waa, Wax, Wya, by, b的字典
        char_to_ix -- 字符映射到索引的字典,用于生成序列的最后加\n以及循环停止条件
        seed -- 随机种子        
    返回：
        indices -- 包含采样字符索引的长度为n的列表。
    """
    
    # 从parameters 中获取参数
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    vocab_size = by.shape[0]  # 也是输入字典的长度 // 2
    n_a = Waa.shape[1]        # 隐层单元的数量
    
    # 步骤1 
    ## 创建独热向量x，初始化为0，后续会根据输出将其设置为独热编码，即被预测值的索引的位置为1
    x = np.zeros((vocab_size,1))
    
    ## 使用0初始化a_prev记忆值
    a_prev = np.zeros((n_a,1))
    
    # 创建索引的空列表，这是包含要生成的字符的索引的列表。
    indices = []
    
    # IDX是检测换行符的标志，我们将其初始化为-1。
    idx = -1
    
    # 循环遍历时间步骤t。在每个时间步中，从概率分布中抽取一个字符，
    # 并将其索引附加到“indices”上，如果我们达到50个字符，
    #（我们应该不太可能有一个训练好的模型），我们将停止循环，这有助于调试并防止进入无限循环
    counter = 0  # 控制随机数种子
    newline_character = char_to_ix["\n"]  # 换行符的索引
    
    while (idx != newline_character and counter < 20):  # 没有检测到换行符并且单词长度小于50
        # 步骤2：使用公式1、2、3进行前向传播
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + ba)  # 下一个时间步的记忆
        z = np.dot(Wya, a) + by
        y = cllm_utils.softmax(z)  # 根据输入来预测输出
        
        # 设定随机种子
        np.random.seed(counter + seed)
        
        # 步骤3：根据概率分布，返回最大概率对应的概率索引
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())
        
        # 添加到索引中
        indices.append(idx)
        
        # 步骤4:将输入字符重写为与采样索引对应的字符。
        x = np.zeros((vocab_size,1))
        x[idx] = 1
        
        # 更新a_prev为a
        a_prev = a 
        
        # 累加器
        seed += 1
        counter +=1
    
    if(counter == 20):
        indices.append(char_to_ix["\n"])
    
    return indices

In [6]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    执行训练模型的单步优化，即一个单词序列对应的索引-->单样本整个时间步的训练
    
    参数：
        X -- 整数列表，其中每个整数映射到词汇表中的字符。
        Y -- 整数列表，与X完全相同，但向左移动了一个索引。
        a_prev -- 上一个记忆
        parameters --   权重、偏置字典，包含了以下参数：
                        Wax -- 权重矩阵乘以输入，维度为(n_a, n_x)
                        Waa -- 权重矩阵乘以隐藏状态，维度为(n_a, n_a)
                        Wya -- 隐藏状态与输出相关的权重矩阵，维度为(n_y, n_a)
                        b -- 偏置，维度为(n_a, 1)
                        by -- 隐藏状态与输出相关的权重偏置，维度为(n_y, 1)
        learning_rate -- 模型学习的速率    
    返回：
        loss -- 损失函数的值（交叉熵损失）
        gradients -- 字典，包含了以下参数：
                        dWax -- 输入到隐藏的权值的梯度，维度为(n_a, n_x)
                        dWaa -- 隐藏到隐藏的权值的梯度，维度为(n_a, n_a)
                        dWya -- 隐藏到输出的权值的梯度，维度为(n_y, n_a)
                        db -- 偏置的梯度，维度为(n_a, 1)
                        dby -- 输出偏置向量的梯度，维度为(n_y, 1)
        a[len(X)-1] -- 最后的隐藏状态，维度为(n_a, 1)
    """
    
    # 前向传播
    loss, cache = cllm_utils.rnn_forward(X, Y, a_prev, parameters)
    
    # 反向传播
    gradients, a = cllm_utils.rnn_backward(X, Y, parameters, cache)
    
    # 梯度修剪，[-5 , 5]
    gradients = clip(gradients,5)
    
    # 更新参数
    parameters = cllm_utils.update_parameters(parameters,gradients,learning_rate)
    
    return loss, gradients, a[len(X)-1]

In [7]:
def model(data, ix_to_char, char_to_ix, num_iterations=3500, 
          n_a=50, dino_names=7,vocab_size=27):
    """
    训练模型并生成恐龙名字
    
    参数：
        data -- 语料库
        ix_to_char -- 索引映射字符字典
        char_to_ix -- 字符映射索引字典
        num_iterations -- 迭代次数
        n_a -- RNN隐层单元数量
        dino_names -- 生成恐龙名字的数量
        vocab_size -- 在文本中的唯一字符的数量
    
    返回：
        parameters -- 学习后了的参数
    """
    
    # 从vocab_size中获取n_x、n_y
    n_x, n_y = vocab_size, vocab_size
    
    # 初始化参数
    parameters = cllm_utils.initialize_parameters(n_a, n_x, n_y)
    
    # 初始化损失-->一个实数
    loss = cllm_utils.get_initial_loss(vocab_size, dino_names)
    
    # 构建恐龙名称列表
    with open("dinos2.txt", encoding='utf-8') as f:
            examples = f.readlines()
            examples = [x.lower().strip() for x in examples]

        # 打乱全部的恐龙名称，返回乱序的恐龙名字列表
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # 初始化记忆为0
    a_prev = np.zeros((n_a,1))
    

    for j in range(num_iterations):        
        index = j % len(examples)  # 达到循环遍历的效果
        X = [None] + [char_to_ix[ch] for ch in examples[index]]  #某单词索引列表
        Y = X[1:] + [char_to_ix["\n"]]  # X左移构成目标索引        
    
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters)
        """
        每次投入一个单词对应的X、Yindex序列进行优化
        """       
        
        # 使用延迟来保持损失平滑,这是为了加速训练。
        loss = cllm_utils.smooth(loss, curr_loss)
        
        # 每2000次迭代，通过sample()生成“\n”字符，检查模型是否学习正确
        if j % 3000 == 0:
            print("第" + str(j+1) + "次迭代，损失值为：" + str(loss))
            
            seed = 0
            for name in range(dino_names):
                # 采样
                sampled_indices = sample(parameters, char_to_ix, seed)
                cllm_utils.print_sample(sampled_indices, ix_to_char)
                
                # 为了得到相同的效果，随机种子+1
                seed += 1
            
            print("\n")
    return parameters

In [9]:
#开始时间
start_time = time.clock()

#开始训练
parameters = model(data, ix_to_char, char_to_ix, num_iterations=15360)

#结束时间
end_time = time.clock()

#计算时差
minium = end_time - start_time

print("执行了：" + str(int(minium / 60)) + "分" + str(int(minium%60)) + "秒")


KeyError: 't'